PA3
Author: Zhuangzhuang Gong, Richard Hua

## 1. Reflection on the annotation task

**Challenges**  
One of the key challenges is that we found it hard to understand the tweets with the culture references and context outside of the tweet itself.  
In this case, we had to make judgement calls,
which made us more aware of the subjectivity involved in annotation. Another takeaway is dealing
with mixed-sentiment tweets. For example, some tweets consist of both positive and negative emotions, making it hard to label them definitively.

**Ideals of improvement**  
- To improve the process of annotation, refine and expand the guidelines of annotation by providing clearer annotation instructions, especially for
mixed-sentiment tweets. This will reduce the subjectivity in the annotation.
- Another way to improve is to introduce InterAnnotator Agreement by having multiple annotators and comparing their results. This not only improve the quality of annotation but also give the insight how subjective the sentiment label is.
- Instead of asking annotators to type in the labels, provide fixed optional labels and guide annotators to select from them to reduce the typo issues.

## 2. Exploring the crowdsourced data


**Read and observe the data**  
As we can see, the sentiment annotations are not expected. There are many inconsistent label formats, such as 'neutral' and 'Neutral' or typos like 'Nutral'. Therefore, we need to clean the data.

In [40]:
import pandas as pd

crowd_train_data = pd.read_csv('crowdsourced_train.csv',sep='\t')
gold_train_data = pd.read_csv('gold_train.csv', sep='\t')
test_data = pd.read_csv('test.csv', sep='\t')
label_counts = crowd_train_data['sentiment'].value_counts()
print(crowd_train_data)

      sentiment                                               text
0      Positive  There's so much misconception on Islam rn so s...
1      Positive  @Mr_Rondeau You should try Iron Maiden at abou...
2      Negative  Going to #FantasticFour tomorrow. Half expecti...
3       Neutral  @cfelan hey hey, just checkng to see if you or...
4      Positive  does anyone just get drunk and watch twilight ...
...         ...                                                ...
10671  Positive  Glad to hear there may be a bigger more public...
10672  Positive  Great stand by the Wolves on 3rd and long. Cur...
10673   Neutral  Ayyye I just purchased my Ed Sheeran tickets f...
10674  Negative  The anti-semitism, the misogyny, and the suppo...
10675  Positive  And yet, I have yet to see the whole series of...

[10676 rows x 2 columns]


**Check the agreement before cleaning the data**  
If we check the inner annotation agreement now by comparing the sentiment labels from the crowdsourced annotator and the gold annotator across 10,675 tweets. The overall accuracy is only 35.63%, and Cohen's Kappa is only 0.19, which indicates a low level of agreement.
This suggests there's either inconsistency or subjective interpretation of sentiment labels.

In [42]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score

accuracy = accuracy_score(crowd_train_data['sentiment'], gold_train_data['sentiment'])
kappa = cohen_kappa_score(crowd_train_data['sentiment'], gold_train_data['sentiment'])
print(f"Cohen's Kappa: {kappa:.2f}")
print(f"Accuracy: {accuracy:.2%}")

Cohen's Kappa: 0.19
Accuracy: 35.63%


**Clean and re-classify labels**  
We easily fixed some inconsistent label formats by converting all labels to lowercase and stripping the blank spaces.  
Furthermore, we found several spelling variations and formatting issues in the label column (e.g., 'nuetral', 'positve', 'nedative', etc.). These were normalised using a mapping dictionary, and all labels were successfully consolidated into the three standard sentiment categories: positive, neutral, and negative.

We corrected the incorrect labels rather than removing them because the quantity of labels is controllable and the labels themselves are distinguishable.

In [44]:
# convert labels to lowercase
crowd_train_data['sentiment'] = crowd_train_data['sentiment'].str.strip().str.lower()
label_counts = crowd_train_data['sentiment'].value_counts()
print("before cleaning")
print(label_counts)

before cleaning
sentiment
neutral           5046
positive          3213
negative          2375
netural             20
nuetral              3
postive              2
postitive            1
neutal               1
npositive            1
neutra l             1
positie              1
negayive             1
nutral               1
neugral              1
negtaive             1
neutrla              1
neutrall             1
neural               1
netutral             1
_x0008_neutral       1
nedative             1
neutral?             1
positve              1
Name: count, dtype: int64


In [45]:
# create a dictionary
label_map = {
    # positive 
    'positve': 'positive',
    'postive': 'positive',
    'postitive': 'positive',
    'positie': 'positive',
    'npositive': 'positive',

    # neutral 
    'neutral?': 'neutral',
    'nuetral': 'neutral',
    '_x0008_neutral': 'neutral',
    'netural': 'neutral',
    'netutral': 'neutral',
    'neural': 'neutral',
    'neutrall': 'neutral',
    'neugral': 'neutral',
    'neutrla': 'neutral',
    'nutral': 'neutral',
    'neutra l': 'neutral',
    'neutal': 'neutral',

    # negative 
    'nedative': 'negative',
    'negtaive': 'negative',
    'negayive': 'negative',
}

label_map.update({
    'positive': 'positive',
    'neutral': 'neutral',
    'negative': 'negative'
})

# apply the dictionary
crowd_train_data['sentiment'] = crowd_train_data['sentiment'].map(label_map)

# check the result
print("After cleaning")
print(crowd_train_data['sentiment'].value_counts())


After cleaning
sentiment
neutral     5079
positive    3219
negative    2378
Name: count, dtype: int64


**Agreement checking after cleaning**  
We evaluated the agreement between the crowd-sourced labels and gold standard labels again across the data after cleaning the labels. The overall accuracy was 65.49% and the Cohen's Kappa was 0.45, indicating 'average' agreement, according to the standard in Richard Johansson's slides.  
This suggests that there's reasonable alignment, while some inconsistency still remains. It's likely due to subjective interpretation of semtimemt tweets.

In [47]:
accuracy = accuracy_score(crowd_train_data['sentiment'], gold_train_data['sentiment'])
kappa = cohen_kappa_score(crowd_train_data['sentiment'], gold_train_data['sentiment'])
print(f"Cohen's Kappa: {kappa:.2f}")
print(f"Accuracy: {accuracy:.2%}")

Cohen's Kappa: 0.45
Accuracy: 65.49%


**Analysis of the annotation distribution**  
We compared the annotation distribution across crowdsourced and gold data. Both showed a similar overall distribution structure, with the majority of labels being 'neutral', followed by 'positive' then 'negative'. However, there's a noticeable difference between 'negative', indicating the sensitivity or interpretation for negative tweets between two groups varies. It suggests that the crowd group is more sensitive to negative sentiment or subjective interpretation of negative tweets, which may contribute to the Cohen's Kappa of 0.45 and the general accuracy of 65.49%.

In [49]:
# Get label counts from each dataset
crowd_counts = crowd_train_data['sentiment'].value_counts()
gold_counts = gold_train_data['sentiment'].value_counts()

# Access values 
crowd_counts_dict = {
    'neutral': crowd_counts.get('neutral', 0),
    'positive': crowd_counts.get('positive', 0),
    'negative': crowd_counts.get('negative', 0)
}

gold_counts_dict = {
    'neutral': gold_counts.get('neutral', 0),
    'positive': gold_counts.get('positive', 0),
    'negative': gold_counts.get('negative', 0)
}

# Create Series
crowd_series = pd.Series(crowd_counts_dict, name='Crowd')
gold_series = pd.Series(gold_counts_dict, name='Gold')

# Combine and analyze
comparison_df = pd.concat([crowd_series, gold_series], axis=1)

# Add percentage comparision
total = comparison_df.sum()
comparison_df['Crowd (%)'] = (comparison_df['Crowd'] / total['Crowd'] * 100).round(2)
comparison_df['Gold (%)'] = (comparison_df['Gold'] / total['Gold'] * 100).round(2)

print(comparison_df)

          Crowd  Gold  Crowd (%)  Gold (%)
neutral    5079  5364      47.57     50.24
positive   3219  3652      30.15     34.21
negative   2378  1660      22.27     15.55


**Reflection**  
#todo

## 3. Implementation of a classifier

Clean tweet

In [53]:
import re
import emoji

def clean_tweet(text):
    # 1. Lowercase
    text = text.lower()
    # 2. Replace URLs
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    # 3. Replace user mentions
    text = re.sub(r'@\w+', ' ', text)
    # 4. Remove hashtags but keep the tag text
    text = re.sub(r'#', '', text)
    # 5. Remove RT (retweet marker)
    text = re.sub(r'\brt\b', ' ', text)
    # 6. Remove emojis or translate them to text
    text = emoji.demojize(text)  # ":smile:" etc
    # 7. Remove non-alphanumeric (keep spaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    # 8. Collapse whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

Pipeline of Clean+TfidfVectorizer+linearSVC

In [55]:
# the actual classification algorithm
from sklearn.svm import LinearSVC
# for converting training and test datasets into matrices
# TfidfVectorizer does this specifically for documents
from sklearn.feature_extraction.text import TfidfVectorizer
# for splitting the dataset into training and test sets 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

X_crowd = crowd_train_data['text'].apply(clean_tweet)
Y_crowd = crowd_train_data['sentiment']

X_crowd_train, X_crowd_eval, Y_crowd_train, Y_crowd_eval = train_test_split(X_crowd, Y_crowd, test_size=0.2, random_state=42)
pipeline = Pipeline([("tfidf", TfidfVectorizer(strip_accents='unicode', sublinear_tf=True)),
                     ("svc", LinearSVC(dual='auto'))])
clf = pipeline.fit(X_crowd_train, Y_crowd_train)
acc = accuracy_score(Y_crowd_eval, clf.predict(X_crowd_eval))
print("acc:",acc)

acc: 0.5730337078651685


To ensure that the classifier has better performance and is more reliable. We used hyperparameter search and cross-validation to select a set of parameters that work better on unknown data in order to maximise the model's performance.

The reason we used hyperparameter search was that it helps improve the predictive performance by finding the optimal settings. Also, it helps balance the model between overfitting and underfitting. On the other hand, cross-validation averages the performance by splitting the data, reducing the variance of the estimation.

Here's a brief about the grid of hyperparameters.  
'tfidf__ngram_range' tells the vectorizer to extract all contiguous word-sequences (n-grams) whose length is between (1,1) and (1,2) and see which one performs better.  
'tfidf__max_df' tells the vectorizer to ignore certain degrees of super-common terms when establishing a dictionary. We searched three common levels of percentage of degrees to filter the noise but also keep the essential terms.   
'tfidf__min_df' tells the vectorizer to ignore certain degrees of rare terms when establishing a dictionary.  We searched three common levels of the amount of rare terms.  
'tfidf__max_features' tells the vectorizer the cap vocabulary size.  We tried two different sizes.  
'svc__C' refers to the punishing coefficient. The smaller the value, the stronger regularisation it has. We set different levels of regularisation to explore the trade-off between overfitting and underfitting.  

In [58]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report

# set up a grid of hyperparameters to search over
param_grid = {
    # TF–IDF params
    'tfidf__ngram_range': [(1,1), (1,2)],    # unigrams vs unigrams+bi-grams
    'tfidf__max_df':       [0.75, 0.85, 1.0], # ignore super-common terms
    'tfidf__min_df':       [3, 5, 10],        # ignore rare terms
    'tfidf__max_features':[10_000, 20_000],   # cap vocabulary size

    # LinearSVC params
    'svc__C':              [0.1, 1.0, 10.0],  # regularization strength
}

# choose a cross-validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# wrap in GridSearchCV
grid = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_weighted',  # or 'accuracy', 'f1_macro' etc.
    n_jobs=-1,              # use all cores
    verbose=1,
)

# fit
grid.fit(X_crowd_train, Y_crowd_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(strip_accents='unicode',
                                                        sublinear_tf=True)),
                                       ('svc', LinearSVC())]),
             n_jobs=-1,
             param_grid={'svc__C': [0.1, 1.0, 10.0],
                         'tfidf__max_df': [0.75, 0.85, 1.0],
                         'tfidf__max_features': [10000, 20000],
                         'tfidf__min_df': [3, 5, 10],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_weighted', verbose=1)

In [59]:
print("Best parameters:\n", grid.best_params_)
print("Best CV score:", grid.best_score_)

# evaluate on held-out test set
y_crowd_pred = grid.predict(X_crowd_eval)
print(classification_report(Y_crowd_eval, y_crowd_pred, digits=4))

Best parameters:
 {'svc__C': 0.1, 'tfidf__max_df': 0.75, 'tfidf__max_features': 10000, 'tfidf__min_df': 10, 'tfidf__ngram_range': (1, 1)}
Best CV score: 0.5469518220767549
              precision    recall  f1-score   support

    negative     0.5760    0.3006    0.3951       479
     neutral     0.5847    0.7792    0.6681      1019
    positive     0.5966    0.4937    0.5403       638

    accuracy                         0.5866      2136
   macro avg     0.5858    0.5245    0.5345      2136
weighted avg     0.5863    0.5866    0.5687      2136



We try gold data next

In [71]:
X_gold = gold_train_data['text'].apply(clean_tweet)
Y_gold = gold_train_data['sentiment']
